![fig](https://raw.githubusercontent.com/zk-ml/demo/main/protocol_overview.jpg)

## zk-ml: truly private machine learning on blockchain

Peiyuan Liao, Milo Cress, @ludens

### **Thesis: modern machine learning platforms lack security and transperancy**

* https://www.kaggle.com/, https://codalab.org/, https://www.crowdai.org/
* Experiences of competing in Kaggle as a Competitions Grandmaster (Peiyuan):
 - user downloads dataset
 - trains model
 - uploads it (???)
 - oranigzer evaluates against some test set (???) 
 - and some metric (???)
 - some random meetings with the organizers (???)
 - prize is delivered 
* https://www.kaggle.com/c/deepfake-detection-challenge/discussion/157983#885598
 - Top ML competition team denied of 500k USD due to opaque data issues
* https://www.kaggle.com/c/global-wheat-detection/discussion/167298
 - Several under-the-hood re-runs, edits to private test set, arbitrarily invalidating submissions
 
### Solution: zk-SNARKS

* ML models usually operates in float, double and half precisions, but zk-SNARKS work in prime fields
* Quantization!
  - Prime complement for negative numbers
  - Slight precision loss on both sides (sometimes better, sometimes worse)
  - Division circuits are very expernsive but doable (negatives, quotioent-remainder relationships)
  - Multiply-add approximations for nonlinearities (sigmoid, tanh, etc.)

### The zk-ml protocol

* What zk-SNARKs provide: 
 - succint proof of computation that neural networks perform certain way under certain datasets
* What ETH provides:
 - securely post and claim bounties
 - reliable way to transfer model for key agreement protocol
 
* The circuit: Linear Regression
    - Checks that all the public inputs that defines a ml dataset (x,y pairs with quantization constants) hashes correctly to hash_input, as this will be the main key for bounty deposit and claim
    - confirms that the actual model params correctly encrypts to the public model params. This way, the bounty issuer can simply use the public key and the encrypted params in the calldata along with their private key to restore the params
    - Performs the model inference and confirms that it indeed has a certain measure wrt public targets.
    - Theory behind GEMM quantization (https://leimao.github.io/article/Neural-Networks-Quantization/)
* The contract
    - Organizers post bounties with IPFS links to datasets
    - Competitors query for bounty and download datasets
    - Competitors trains model, quantizes it and generate proofs locally
    - (if conditions are met) Competitors uploads proof and claims bounty
    - (if conditions are not met) Organizers removes bounty and gets fund back

### Future works

* More ML models
 - Language models: I-BERT (https://arxiv.org/abs/2101.01321)
 - Gradient Boosted Decision Trees, XGBoost, LightGBM, etc: trees are representable as circuits
 - Compiler from neural network IR to groth16 verifier (zokrates w. optimizations)
* Protocol revamp
 - Multiple contributions to a contract: proxy contracts
 - Pushing beyond contract size limit:
     - Diamond patterns, libraries
     - 16/8/4-bit quantization and bit packing inside uint256
     - Moving zk-SNARK computation to cryptoprocessors
 - Preventing frontrunners: bounty issuer can see the calldata, front-run the transcation, and remove the bounty
 - Preventing over-fitting for over-parameterized models (MLP, ConvNets): two stage competition -> public leaderboard proof and private leaderboard proofs
 - Preventing adversarial organizers creating invalid test sets (random noise, exceptionally hard, etc).
     - User public keys are kept private during public leaderboard
     - Slashing if organizer does not relesase private dataset in-time
     - Slashing if private dataset behaves in an adversarial manner 
* DAO
 

In [1]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [2]:
alias prepare source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml add_bounty --amount 1 

In [3]:
prepare

Hashed inputs: 
14797455496207951391356508759149962584765968173479481191220882411966396840571n
Your Public Key: 
[
  1201985518978813069048614242908853769836641152766343625459923948687381884866n,
  19496081506364456486564631647699620263934626467785977126387698776414945878171n
]
Your Private Key: 
3078011588103099000651955808511951698555472462434891497141660669970337258902n
IPFS available at https://ipfs.io/ipfs/QmWLRJVL5uViT7h64bdeUM3GKMWP9DSWRggGC8igDuQdHR
Current Balance
109.616883399991896552


In [4]:
zkml list_datasets

Available datasets:
[
  '14797455496207951391356508759149962584765968173479481191220882411966396840571'
]


In [5]:
zkml list_bounties --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571'

Available bounties on dataset: 14797455496207951391356508759149962584765968173479481191220882411966396840571
[
  {
    PubKey1: '1201985518978813069048614242908853769836641152766343625459923948687381884866',
    PubKey2: '19496081506364456486564631647699620263934626467785977126387698776414945878171',
    MSEcap: '12888',
    Bounty: '1.0',
    Issuer: '0xd3162F2B88d05C882a1B26031E144753337ACDBF',
    IPFS: 'QmWLRJVL5uViT7h64bdeUM3GKMWP9DSWRggGC8igDuQdHR'
  }
]


Command:

```
zkml download_dataset --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571' --publickey ./keys/out_public.json --mse 12888 --path ./ipfs_dataset
```

In [7]:
zkml download_dataset --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571' --publickey ./keys/out_public.json --mse 12888 --path ./ipfs_dataset

In [8]:
!ls ./ipfs_dataset

X.npy Y.npy


In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [10]:
X = np.load('./ipfs_dataset/X.npy')
Y = np.load('./ipfs_dataset/Y.npy')

In [11]:
Y.shape, X.shape

((20, 1), (20, 4))

In [14]:
# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X, Y)

# Making Predictions
pred = lr.predict(X)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(Y, pred))

mse = mean_squared_error(Y, pred)

Mean Squared Error: 0.07865899873764468


In [17]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
Yt_expected = Y.reshape(-1, 1)

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('dataset/X.npy',X)
np.save('dataset/Y.npy',Yt_expected)

In [20]:
shuffle = lambda x: np.random.shuffle(x)
shuffle(W)
shuffle(b)
print(mean_squared_error(Y, np.matmul(X, W.reshape(-1)) + b.reshape(-1)))

np.save('model_shuffled/W.npy',W)
np.save('model_shuffled/b.npy',b)

0.658287887486153


In [21]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = 0.07864
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

Command:

```
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json
```

In [22]:
zkml claim_bounty --payment 0xd3162F2B88d05C882a1B26031E144753337ACDBF --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.658287887486153
... quantized  65536
Mean Squared Error simulated:  0.65811511933101396465
... quantized  65536
Circuit Outputs:
107823
Proof took 17.329 s
ERROR: Invalid proof
An unexpected error occurred:

Error: Could not verify the proof
    at SimpleTaskDefinition.action (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/hardhat.config.js:273:26)
    at Environment._runTaskDefinition (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:217:14)
    at Environment.run (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:129:14)
    at main (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/cli/cli.ts:197:5)


Command:

```
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json
```

In [23]:
zkml claim_bounty --payment 0xd3162F2B88d05C882a1B26031E144753337ACDBF --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.0786589987376446
... quantized  12891
Mean Squared Error simulated:  0.07864249526948667579
... quantized  12888
Circuit Outputs:
12888
Proof took 16.409 s
INFO: OK!
Paying 0xd3162F2B88d05C882a1B26031E144753337ACDBF
With balance
108.616501100991896552
Your Public Key: 
[
  4842771413884360979635609941922451201670854864962458988387057397512039623940n,
  313941953243899344658502354250141108878460697882164557538046223348611775672n
]
Your Private Key: 
5581888367090473215770974626509128864134834921982778138974737194930113876268n
Current Balance
108.616501100991896552


In [24]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: 14797455496207951391356508759149962584765968173479481191220882411966396840571
[]


In [25]:
zkml list_datasets

Available datasets:
[]
